1. 데이터 셋 불러오고 정규화하기
[출처] 파이토치 딥러닝 튜토리얼 예제 Transfer Learning(전이 학습)을 이용한 CIFAR-10|작성자 ckdgus1433

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

transform = transforms.Compose([
        transforms.Resize(224, interpolation=2),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 40

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset=torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

dataloaders = {'train' : trainloader, 'val' : testloader }
dataset_sizes = { 'train' : len(trainset) , 'val' : len(testset) }

class_names = ('male','female')
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #GPU 사용가능 여부확인

Files already downloaded and verified
Files already downloaded and verified
('male', 'female')


In [27]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(trainloader)
images, labels = dataiter.next()
#print(tf.size(images))
# 이미지 보여주기
imshow(torchvision.utils.make_grid(images))
# 정답(label) 출력
print(' '.join('%5s' % class_names[labels[j]] for j in range(4)))
plt.show()

IndexError: tuple index out of range

2. 모델 정의하기

In [8]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10).cuda()

model_ft = model_ft.to(device)

3. 손실함수와 Optimizer 정의하기

In [9]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

4. Training set을 사용하여 신경망 학습하기 &

5. Test set을 사용하여 신경망이 잘 훈련 했는지를 검사하기

In [10]:

def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time() #시작 시간을 기록(총 소요 시간 계산을 위해)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1)) #epoch를 카운트
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:  #train mode와 validation mode 순으로 진행
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # dataloader로부터 dataset과 그에 해당되는 label을 불러온다.
            for inputs, labels in dataloaders[phase]:  
                inputs = inputs.to(device) #GPU로 입력데이터를 올림
                labels = labels.to(device) #GPU로 label을 올림
                # zero the parameter gradients
                optimizer.zero_grad()  #Gradient를 0으로 초기화

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs).cuda()
                    _, preds = torch.max(outputs, 1) # 마지막 layer에서 가장 값이 큰 1개의 class를 예측 값으로 지정
                    loss = criterion(outputs, labels).cuda()

                    # backward + optimize only if in training phase
                    if phase == 'train': # training 모드에서는 weight를 update한다.
                        loss.backward()
                        optimizer.step()
                    else:
                        preds=preds.cpu()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                preds = preds.cuda()
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


print('================Finished modeling')

================Finished modeling


In [11]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=5)

Epoch 0/19
----------
train Loss: 0.4259 Acc: 0.8597
val Loss: 0.2042 Acc: 0.9325

Epoch 1/19
----------
train Loss: 0.1582 Acc: 0.9483
val Loss: 0.1720 Acc: 0.9417

Epoch 2/19
----------
train Loss: 0.0891 Acc: 0.9714
val Loss: 0.1544 Acc: 0.9493

Epoch 3/19
----------
train Loss: 0.0477 Acc: 0.9867
val Loss: 0.1544 Acc: 0.9503

Epoch 4/19
----------
train Loss: 0.0290 Acc: 0.9926
val Loss: 0.1537 Acc: 0.9496

Epoch 5/19
----------
train Loss: 0.0176 Acc: 0.9963
val Loss: 0.1579 Acc: 0.9526

Epoch 6/19
----------
train Loss: 0.0108 Acc: 0.9984
val Loss: 0.1521 Acc: 0.9536

Epoch 7/19
----------
train Loss: 0.0097 Acc: 0.9987
val Loss: 0.1521 Acc: 0.9543

Epoch 8/19
----------
train Loss: 0.0083 Acc: 0.9991
val Loss: 0.1540 Acc: 0.9543

Epoch 9/19
----------
train Loss: 0.0082 Acc: 0.9990
val Loss: 0.1553 Acc: 0.9537

Epoch 10/19
----------
train Loss: 0.0074 Acc: 0.9993
val Loss: 0.1534 Acc: 0.9540

Epoch 11/19
----------
train Loss: 0.0070 Acc: 0.9993
val Loss: 0.1543 Acc: 0.9545

Ep